# Импорт необходимых библиотек

In [23]:
!pip install dostoevsky

  Using cached pybind11-2.10.4-py3-none-any.whl (222 kB)
  Running setup.py clean for fasttext
Failed to build fasttext
    Running setup.py install for fasttext: started
    Running setup.py install for fasttext: finished with status 'error'


  ERROR: Command errored out with exit status 1:
   command: 'C:\ProgramData\Anaconda3\python.exe' -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Temp\\pip-install-wcyfwoft\\fasttext_0c109378270f49c39b4a1717b03af06e\\setup.py'"'"'; __file__='"'"'C:\\Temp\\pip-install-wcyfwoft\\fasttext_0c109378270f49c39b4a1717b03af06e\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Temp\pip-wheel-nxicduxs'
       cwd: C:\Temp\pip-install-wcyfwoft\fasttext_0c109378270f49c39b4a1717b03af06e\
  Complete output (18 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.8
  creating build\lib.win-amd64-3.8\fasttext
  copying python\fasttext_module\fasttext\FastText.py -> build\lib.win-amd64-3.8\fasttext
  copying python\fasttext_module\fasttext\__init__.py -> build\lib.win-amd64-3.8\fastt

In [1]:
import pandas as pd
import nltk
from nltk.corpus import brown
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
from gensim.models import Word2Vec as wv
from gensim.models import KeyedVectors
import pymorphy2
from pymorphy2 import MorphAnalyzer
import emoji
import numpy as np

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Максим\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Датасет ВК

## Загрузка датасета

In [7]:
colnames = ['comment',]

In [14]:
comments = pd.read_csv('datasets/vk.csv', names=colnames, header=0, sep='\t')

In [15]:
comments

,comment
0,['норма']
1,['😁']
2,"['воронка', 'цветок', 'вода', '💮']"
3,"['сша', 'фашизм']"
4,"['злодей', 'целый', 'месяц', 'ждать']"
...,...
20575,"['с', 'фамилия', '🙄']"
20576,"['наш', 'день', 'повезти', 'мужик', 'фамилия',..."
20577,"['чудо', 'чудной', 'диво', 'дивный']"
20578,"['пригреться', 'уснуть', 'свариться', 'лягушка..."


## Преобразование датасета к формату списка списков для обучения модели w2v

In [10]:
def str_to_list(i):
    new_list = eval(i)
    return new_list

In [16]:
comments_str = pd.DataFrame()

In [17]:
comments_str['comment'] = comments['comment'].apply(str_to_list)

In [18]:
vk_data = comments['comment'].values.tolist()

In [19]:
vk_data[:5]

["['норма']",
 "['😁']",
 "['воронка', 'цветок', 'вода', '💮']",
 "['сша', 'фашизм']",
 "['злодей', 'целый', 'месяц', 'ждать']"]

# Подготовка датасета для анализа тональности  *(дополнительно)* 

In [20]:
negative = ['😡', '👿', '😾', '😤', '😠', '🤬', '💀', '☠️']

In [21]:
sarcastic = ['💩', '🤡', '🙃', '🤢', '😈']  # Нужно дополнить

In [22]:
positive = ['😀', '😃', '😉']  # Нужно дополнить